In [31]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

import matplotlib.pyplot as plt

from datetime import datetime

In [32]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# importing libraries for Geographic and Interactive visualizations
from folium import Choropleth, Circle, Marker, Map
from folium.plugins import HeatMap, MarkerCluster
import plotly.express as px
import plotly.graph_objects as go
import math


# Boston Crime Data

In [33]:
bo_df = pd.read_csv('https://capstone-crime-bucket.s3.amazonaws.com/crime.csv', encoding='latin-1')

bo_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [34]:
print("Total number of crimes in the dataset: {}".format(len(bo_df)))
bo_df.head()

Total number of crimes in the dataset: 319073


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [35]:
bo_df.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA          object
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [36]:
for col in bo_df.columns:
        print(col)

INCIDENT_NUMBER
OFFENSE_CODE
OFFENSE_CODE_GROUP
OFFENSE_DESCRIPTION
DISTRICT
REPORTING_AREA
SHOOTING
OCCURRED_ON_DATE
YEAR
MONTH
DAY_OF_WEEK
HOUR
UCR_PART
STREET
Lat
Long
Location


In [37]:
crimeByType = bo_df['OFFENSE_DESCRIPTION'].value_counts()
crimeByType

SICK/INJURED/MEDICAL - PERSON                  18783
INVESTIGATE PERSON                             18754
M/V - LEAVING SCENE - PROPERTY DAMAGE          16323
VANDALISM                                      15154
ASSAULT SIMPLE - BATTERY                       14791
                                               ...  
KILLING OF FELON BY POLICE                         1
AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON         1
DRUGS - GLUE INHALATION                            1
ROBBERY - KNIFE - STREET                           1
DRUGS - POSS CLASS D - INTENT MFR DIST DISP        1
Name: OFFENSE_DESCRIPTION, Length: 244, dtype: int64

In [38]:
crimeByCode = bo_df['OFFENSE_CODE'].value_counts()
crimeByCode

3006    18783
3115    18754
3831    16323
1402    15154
802     14799
        ...  
404         1
112         1
2609        1
2606        1
1864        1
Name: OFFENSE_CODE, Length: 222, dtype: int64

In [39]:
bo_df.nunique()

INCIDENT_NUMBER        282517
OFFENSE_CODE              222
OFFENSE_CODE_GROUP         67
OFFENSE_DESCRIPTION       244
DISTRICT                   12
REPORTING_AREA            879
SHOOTING                    1
OCCURRED_ON_DATE       233229
YEAR                        4
MONTH                      12
DAY_OF_WEEK                 7
HOUR                       24
UCR_PART                    4
STREET                   4657
Lat                     18178
Long                    18178
Location                18194
dtype: int64

In [40]:
bo_df.DISTRICT.value_counts()

B2     49945
C11    42530
D4     41915
A1     35717
B3     35442
C6     23460
D14    20127
E13    17536
E18    17348
A7     13544
E5     13239
A15     6505
Name: DISTRICT, dtype: int64

In [41]:
bo_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [42]:
bo_df.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,319073.000000,319073.000000,319073.000000,319073.000000,299074.000000,299074.000000
mean,2317.546956,2016.560586,6.609719,13.118205,42.214381,-70.908272
std,1185.285543,0.996344,3.273691,6.294205,2.159766,3.493618
min,111.000000,2015.000000,1.000000,0.000000,-1.000000,-71.178674
25%,1001.000000,2016.000000,4.000000,9.000000,42.297442,-71.097135
50%,2907.000000,2017.000000,7.000000,14.000000,42.325538,-71.077524
75%,3201.000000,2017.000000,9.000000,18.000000,42.348624,-71.062467
max,3831.000000,2018.000000,12.000000,23.000000,42.395042,-1.000000


In [43]:
group_crime = bo_df.groupby('OFFENSE_DESCRIPTION')
group_crime = bo_df['OFFENSE_DESCRIPTION'].value_counts()
group_crime

SICK/INJURED/MEDICAL - PERSON                  18783
INVESTIGATE PERSON                             18754
M/V - LEAVING SCENE - PROPERTY DAMAGE          16323
VANDALISM                                      15154
ASSAULT SIMPLE - BATTERY                       14791
                                               ...  
KILLING OF FELON BY POLICE                         1
AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON         1
DRUGS - GLUE INHALATION                            1
ROBBERY - KNIFE - STREET                           1
DRUGS - POSS CLASS D - INTENT MFR DIST DISP        1
Name: OFFENSE_DESCRIPTION, Length: 244, dtype: int64

In [44]:
group_crime.describe()

count      244.000000
mean      1307.676230
std       3053.979367
min          1.000000
25%          6.750000
50%        188.000000
75%        963.000000
max      18783.000000
Name: OFFENSE_DESCRIPTION, dtype: float64

In [45]:
#find null values
for column in bo_df.columns:
    print(f"Column {column} has {bo_df[column].isnull().sum()} null values")

Column INCIDENT_NUMBER has 0 null values
Column OFFENSE_CODE has 0 null values
Column OFFENSE_CODE_GROUP has 0 null values
Column OFFENSE_DESCRIPTION has 0 null values
Column DISTRICT has 1765 null values
Column REPORTING_AREA has 0 null values
Column SHOOTING has 318054 null values
Column OCCURRED_ON_DATE has 0 null values
Column YEAR has 0 null values
Column MONTH has 0 null values
Column DAY_OF_WEEK has 0 null values
Column HOUR has 0 null values
Column UCR_PART has 90 null values
Column STREET has 10871 null values
Column Lat has 19999 null values
Column Long has 19999 null values
Column Location has 0 null values


In [46]:
bo_df.dropna(subset = ['DISTRICT', 'Lat', 'Long'], how='all', inplace=True)


In [47]:
bo_df.drop(columns=["INCIDENT_NUMBER", "OFFENSE_CODE_GROUP", "REPORTING_AREA", "SHOOTING", "MONTH", "DAY_OF_WEEK", "HOUR", "UCR_PART", "STREET"], inplace = True)
bo_df.head(50)

,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,OCCURRED_ON_DATE,YEAR,Lat,Long,Location
0,619,LARCENY ALL OTHERS,D14,2018-09-02 13:00:00,2018,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,1402,VANDALISM,C11,2018-08-21 00:00:00,2018,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,3410,TOWED MOTOR VEHICLE,D4,2018-09-03 19:27:00,2018,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,3114,INVESTIGATE PROPERTY,D4,2018-09-03 21:16:00,2018,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,3114,INVESTIGATE PROPERTY,B3,2018-09-03 21:05:00,2018,42.275365,-71.090361,"(42.27536542, -71.09036101)"
5,3820,M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY,C11,2018-09-03 21:09:00,2018,42.290196,-71.071590,"(42.29019621, -71.07159012)"
6,724,AUTO THEFT,B2,2018-09-03 21:25:00,2018,42.306072,-71.082733,"(42.30607218, -71.08273260)"
7,3301,VERBAL DISPUTE,B2,2018-09-03 20:39:37,2018,42.327016,-71.105551,"(42.32701648, -71.10555088)"
8,301,ROBBERY - STREET,C6,2018-09-03 20:48:00,2018,42.331521,-71.070853,"(42.33152148, -71.07085307)"
9,3301,VERBAL DISPUTE,C11,2018-09-03 20:38:00,2018,42.295147,-71.058608,"(42.29514664, -71.05860832)"


In [48]:
# Drop null rows
bo_clean = bo_df.dropna()

In [49]:
bo_clean["City"] = 1
bo_clean.head()

/Users/rotation/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,OCCURRED_ON_DATE,YEAR,Lat,Long,Location,City
0,619,LARCENY ALL OTHERS,D14,2018-09-02 13:00:00,2018,42.357791,-71.139371,"(42.35779134, -71.13937053)",1
1,1402,VANDALISM,C11,2018-08-21 00:00:00,2018,42.306821,-71.060300,"(42.30682138, -71.06030035)",1
2,3410,TOWED MOTOR VEHICLE,D4,2018-09-03 19:27:00,2018,42.346589,-71.072429,"(42.34658879, -71.07242943)",1
3,3114,INVESTIGATE PROPERTY,D4,2018-09-03 21:16:00,2018,42.334182,-71.078664,"(42.33418175, -71.07866441)",1
4,3114,INVESTIGATE PROPERTY,B3,2018-09-03 21:05:00,2018,42.275365,-71.090361,"(42.27536542, -71.09036101)",1


In [50]:
bo_clean = bo_clean.reset_index(drop=True)

In [51]:
bo_clean.to_csv("https://capstone-crime-bucket.s3.amazonaws.com/crime.csv",header=True)

# Los Angeles Crime Data


In [52]:
la_df = pd.read_csv('https://capstone-crime-bucket.s3.amazonaws.com/Crimes_2012-2016.csv', encoding='latin-1')

la_df.head()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013


In [53]:
print("Total number of crimes in the dataset: {}".format(len(la_df)))
la_df.head()

Total number of crimes in the dataset: 1132034


,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013


In [54]:
for col in la_df.columns:
        print(col)

Unnamed: 0
DR.NO
DATE.OCC
TIME.OCC
AREA
AREA.NAME
RD
Crm.Cd
CrmCd.Desc
Location.1
lat
long
Year


In [55]:
crimeByType = la_df['CrmCd.Desc'].value_counts()
crimeByType

TRAFFIC DR #                               164915
BATTERY - SIMPLE ASSAULT                    89993
VEHICLE - STOLEN                            73127
BURGLARY FROM VEHICLE                       72291
BURGLARY                                    69339
                                            ...  
TILL TAP - GRAND THEFT ($950.01 & OVER)         3
MANSLAUGHTER, NEGLIGENT                         2
BLOCKING DOOR INDUCTION CENTER                  2
THEFT, COIN MACHINE - GRAND                     1
HOMICIDE (NON-UCR)                              1
Name: CrmCd.Desc, Length: 161, dtype: int64

In [56]:
la_df.dtypes

Unnamed: 0      int64
DR.NO           int64
DATE.OCC       object
TIME.OCC        int64
AREA            int64
AREA.NAME      object
RD              int64
Crm.Cd          int64
CrmCd.Desc     object
Location.1     object
lat           float64
long          float64
Year            int64
dtype: object

In [57]:
crimeByCode = la_df['Crm.Cd'].value_counts()
crimeByCode

997    164915
624     89993
440     74815
510     73127
330     72291
        ...  
347         4
470         3
113         2
432         2
111         1
Name: Crm.Cd, Length: 136, dtype: int64

In [58]:
la_df.nunique()

Unnamed: 0    1132034
DR.NO         1131013
DATE.OCC         1770
TIME.OCC         1439
AREA               21
AREA.NAME          21
RD               1303
Crm.Cd            136
CrmCd.Desc        161
Location.1      68551
lat              5440
long             5139
Year                5
dtype: int64

In [59]:
la_df.dropna()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132029,1132029,162010003,04/30/2016,2200,20,Olympic,2069,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...","(34.0508, -118.2877)",34.0508,-118.2877,2016
1132030,1132030,160315990,06/29/2016,1020,3,Southwest,353,946,OTHER MISCELLANEOUS CRIME,"(34.0199, -118.3363)",34.0199,-118.3363,2016
1132031,1132031,161200696,05/01/2016,2030,12,77th Street,1256,626,SPOUSAL(COHAB) ABUSE - SIMPLE ASSAULT,"(33.9717, -118.2893)",33.9717,-118.2893,2016
1132032,1132032,161015111,09/05/2016,800,10,West Valley,1017,440,THEFT PLAIN - PETTY ($950 & UNDER),"(34.2067, -118.5014)",34.2067,-118.5014,2016


In [138]:
la_clean.drop(['Unnamed: 0', 'DR.NO', 'TIME.OCC', 'RD', 'Status', 'Status.Desc', 'LOCATION', 'Cross.Street', 'AREA', 'Neighborhood'], axis = 1, inplace = True)

la_clean.head()

,CrimeDate,CrimeCode,Description,Location,Latitude,Longitude,Year,City
0,03/20/2013,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013,2
1,03/10/2013,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013,2
2,12/18/2013,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013,2
3,10/18/2013,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013,2
4,05/25/2013,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013,2


In [61]:
la_df.describe()

,Unnamed: 0,DR.NO,TIME.OCC,AREA,RD,Crm.Cd,lat,long,Year
count,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06,1.132034e+06
mean,5.660165e+05,1.405632e+08,1.367111e+03,1.110129e+01,1.156419e+03,5.791831e+02,3.381956e+01,-1.174601e+02,2.013932e+03
std,3.267902e+05,1.391008e+07,6.375322e+02,5.983360e+00,5.985015e+02,2.589706e+02,2.962788e+00,1.028295e+01,1.397677e+00
min,0.000000e+00,4.180000e+02,1.000000e+00,1.000000e+00,1.000000e+02,1.100000e+02,0.000000e+00,-1.188551e+02,2.012000e+03
25%,2.830082e+05,1.306045e+08,9.300000e+02,6.000000e+00,6.450000e+02,3.410000e+02,3.400940e+01,-1.184378e+02,2.013000e+03
50%,5.660165e+05,1.410111e+08,1.430000e+03,1.100000e+01,1.181000e+03,5.100000e+02,3.406200e+01,-1.183308e+02,2.014000e+03
75%,8.490248e+05,1.514144e+08,1.900000e+03,1.600000e+01,1.665000e+03,7.450000e+02,3.417490e+01,-1.182782e+02,2.015000e+03
max,1.132033e+06,1.699148e+08,2.359000e+03,2.100000e+01,2.198000e+03,9.970000e+02,3.480870e+01,0.000000e+00,2.016000e+03


In [62]:
#find null values
for column in la_df.columns:
    print(f"Column {column} has {la_df[column].isnull().sum()} null values")

Column Unnamed: 0 has 0 null values
Column DR.NO has 0 null values
Column DATE.OCC has 0 null values
Column TIME.OCC has 0 null values
Column AREA has 0 null values
Column AREA.NAME has 0 null values
Column RD has 0 null values
Column Crm.Cd has 0 null values
Column CrmCd.Desc has 0 null values
Column Location.1 has 0 null values
Column lat has 0 null values
Column long has 0 null values
Column Year has 0 null values


In [63]:
# Drop null rows
la_clean = la_df.dropna()

In [64]:
la_clean.head()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013


In [65]:
import ast
la_clean['Location.1'] = la_clean['Location.1'].apply(ast.literal_eval)

In [66]:
la_clean['lat'] = la_clean['Location.1'].str[0]
la_clean['long'] = la_clean['Location.1'].str[1]

In [67]:
la_clean["City"] = 2
la_clean.head()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year,City
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013,2
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013,2
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013,2
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013,2
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013,2


In [68]:
la_clean.head()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year,City
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013,2
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013,2
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013,2
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013,2
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013,2


In [69]:
la_clean['Year'] = pd.DatetimeIndex(la_clean['DATE.OCC']).year
la_clean.head()

,Unnamed: 0,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Location.1,lat,long,Year,City
0,0,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,"(34.0776, -118.308)",34.0776,-118.3080,2013,2
1,1,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,"(34.1113, -118.3336)",34.1113,-118.3336,2013,2
2,2,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,"(33.9406, -118.2338)",33.9406,-118.2338,2013,2
3,3,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,"(33.9449, -118.2332)",33.9449,-118.2332,2013,2
4,4,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),"(33.8135, -118.2992)",33.8135,-118.2992,2013,2


In [70]:
la_clean = la_clean.reset_index(drop=True)

In [71]:
la_clean.to_csv("https://capstone-crime-bucket.s3.amazonaws.com/Crimes_2012-2016.csv",header=True)

# Baltimore Crime Data

In [72]:
bal_df = pd.read_csv('https://capstone-crime-bucket.s3.amazonaws.com/Baltimore_Crime_Data.csv', encoding='latin-1')

bal_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
1,09/02/2017,23:00:00,7A,800 NEWINGTON AVE,AUTO THEFT,O,NaN,133.0,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",STREET,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1


In [73]:
print("Total number of crimes in the dataset: {}".format(len(bal_df)))
bal_df.head()

Total number of crimes in the dataset: 276529


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
1,09/02/2017,23:00:00,7A,800 NEWINGTON AVE,AUTO THEFT,O,NaN,133.0,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",STREET,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1


In [74]:
for col in bal_df.columns:
        print(col)

CrimeDate
CrimeTime
CrimeCode
Location
Description
Inside/Outside
Weapon
Post
District
Neighborhood
Longitude
Latitude
Location 1
Premise
Total Incidents


In [75]:
crimeByType = bal_df['Description'].value_counts()
crimeByType

LARCENY                 60528
COMMON ASSAULT          45518
BURGLARY                42538
LARCENY FROM AUTO       36295
AGG. ASSAULT            27513
AUTO THEFT              26838
ROBBERY - STREET        17691
ROBBERY - COMMERCIAL     4141
ASSAULT BY THREAT        3503
SHOOTING                 2910
ROBBERY - RESIDENCE      2866
RAPE                     1637
HOMICIDE                 1559
ROBBERY - CARJACKING     1528
ARSON                    1464
Name: Description, dtype: int64

In [76]:
bal_df.dtypes

CrimeDate           object
CrimeTime           object
CrimeCode           object
Location            object
Description         object
Inside/Outside      object
Weapon              object
Post               float64
District            object
Neighborhood        object
Longitude          float64
Latitude           float64
Location 1          object
Premise             object
Total Incidents      int64
dtype: object

In [77]:
crimeByCode = bal_df['CrimeCode'].value_counts()
crimeByCode

4E     45518
6D     36295
5A     25699
7A     25230
6G     15920
       ...  
3N         5
8CV        4
8DO        2
3LK        2
6K         1
Name: CrimeCode, Length: 81, dtype: int64

In [78]:
bal_df.nunique()

CrimeDate           2072
CrimeTime           3017
CrimeCode             81
Location           26008
Description           15
Inside/Outside         4
Weapon                 4
Post                 179
District               9
Neighborhood         278
Longitude          17678
Latitude           13507
Location 1         95497
Premise              123
Total Incidents        1
dtype: int64

In [79]:
bal_df.dropna()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1
6,09/02/2017,21:15:00,1F,3400 HARMONY CT,HOMICIDE,Outside,FIREARM,232.0,SOUTHEASTERN,Canton,-76.56827,39.28202,"(39.2820200000, -76.5682700000)",Street,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276504,01/01/2012,01:10:00,4B,2200 EASTERN AV,AGG. ASSAULT,O,KNIFE,212.0,SOUTHEASTERN,Upper Fells Point,-76.58572,39.28593,"(39.2859300000, -76.5857200000)",STREET,1
276515,01/01/2012,00:40:00,4E,600 N BROADWAY,COMMON ASSAULT,I,HANDS,321.0,EASTERN,Dunbar-Broadway,-76.59403,39.29737,"(39.2973700000, -76.5940300000)",HOSP/NURS.,1
276516,01/01/2012,00:38:00,4E,3400 ELMLEY AV,COMMON ASSAULT,I,HANDS,434.0,NORTHEASTERN,Belair-Edison,-76.57521,39.31702,"(39.3170200000, -76.5752100000)",ROW/TOWNHO,1
276517,01/01/2012,00:30:00,4E,1100 S CHARLES ST,COMMON ASSAULT,O,HANDS,942.0,SOUTHERN,Federal Hill,-76.61440,39.27678,"(39.2767800000, -76.6144000000)",STREET,1


In [80]:
bal_df.describe()

,Post,Longitude,Latitude,Total Incidents
count,276305.000000,274325.000000,274325.000000,276529.0
mean,506.256821,-76.617444,39.307399,1.0
std,260.735370,0.042306,0.029556,0.0
min,2.000000,-76.711620,39.200410,1.0
25%,243.000000,-76.648460,39.288330,1.0
50%,511.000000,-76.613960,39.303640,1.0
75%,731.000000,-76.587440,39.327810,1.0
max,945.000000,-76.528500,39.372930,1.0


In [141]:
bal_clean.drop(['Total Incidents','Weapon', 'Post', 'Premise', 'Inside/Outside', 'Location', 'CrimeTime', 'Neighborhood', 'District'], axis = 1, inplace = True)
bal_clean.head()

,CrimeDate,CrimeCode,Description,Longitude,Latitude,Location,City,Year
0,09/02/2017,3JK,ROBBERY - RESIDENCE,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",3,2017
1,09/02/2017,7A,AUTO THEFT,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",3,2017
2,09/02/2017,9S,SHOOTING,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",3,2017
3,09/02/2017,4C,AGG. ASSAULT,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",3,2017
4,09/02/2017,4E,COMMON ASSAULT,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",3,2017


In [101]:
#find null values
for column in bal_df.columns:
    print(f"Column {column} has {bal_df[column].isnull().sum()} null values")

Column CrimeDate has 0 null values
Column CrimeTime has 0 null values
Column CrimeCode has 0 null values
Column Description has 0 null values
Column District has 80 null values
Column Neighborhood has 2740 null values
Column Longitude has 2204 null values
Column Latitude has 2204 null values
Column Location 1 has 2204 null values


In [102]:
# Drop null rows
bal_clean = bal_df.dropna()

In [103]:
bal_clean["City"] = 3
bal_clean.head()

/Users/rotation/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CrimeDate,CrimeTime,CrimeCode,Description,District,Neighborhood,Longitude,Latitude,Location 1,City
0,09/02/2017,23:30:00,3JK,ROBBERY - RESIDENCE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",3
1,09/02/2017,23:00:00,7A,AUTO THEFT,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",3
2,09/02/2017,22:53:00,9S,SHOOTING,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",3
3,09/02/2017,22:50:00,4C,AGG. ASSAULT,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",3
4,09/02/2017,22:31:00,4E,COMMON ASSAULT,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",3


In [104]:
bal_clean.head()

,CrimeDate,CrimeTime,CrimeCode,Description,District,Neighborhood,Longitude,Latitude,Location 1,City
0,09/02/2017,23:30:00,3JK,ROBBERY - RESIDENCE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",3
1,09/02/2017,23:00:00,7A,AUTO THEFT,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",3
2,09/02/2017,22:53:00,9S,SHOOTING,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",3
3,09/02/2017,22:50:00,4C,AGG. ASSAULT,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",3
4,09/02/2017,22:31:00,4E,COMMON ASSAULT,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",3


In [105]:
bal_clean['Year'] = pd.DatetimeIndex(bal_clean['CrimeDate']).year
bal_clean.head()

/Users/rotation/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CrimeDate,CrimeTime,CrimeCode,Description,District,Neighborhood,Longitude,Latitude,Location 1,City,Year
0,09/02/2017,23:30:00,3JK,ROBBERY - RESIDENCE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",3,2017
1,09/02/2017,23:00:00,7A,AUTO THEFT,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",3,2017
2,09/02/2017,22:53:00,9S,SHOOTING,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",3,2017
3,09/02/2017,22:50:00,4C,AGG. ASSAULT,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",3,2017
4,09/02/2017,22:31:00,4E,COMMON ASSAULT,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",3,2017


In [106]:
bal_clean = bal_clean.reset_index(drop=True)

In [107]:
bal_clean.to_csv("https://capstone-crime-bucket.s3.amazonaws.com/Baltimore_Crime_Data.csv",header=True)

# Merge Datasets

In [143]:
bo_clean.columns.tolist()

['CrimeCode',
 'Description',
 'DISTRICT',
 'CrimeDate',
 'Year',
 'Latitude',
 'Longitude',
 'Location',
 'City']

In [144]:
la_clean.columns.tolist()

['CrimeDate',
 'CrimeCode',
 'Description',
 'Location',
 'Latitude',
 'Longitude',
 'Year',
 'City']

In [145]:
bal_clean.columns.tolist()

['CrimeDate',
 'CrimeCode',
 'Description',
 'Longitude',
 'Latitude',
 'Location',
 'City',
 'Year']

In [146]:
 
bo_clean.rename(columns={'OFFENSE_CODE': 'CrimeCode', 'YEAR': 'Year', 'Lat': 'Latitude', 'Long': 'Longitude', 'OCCURRED_ON_DATE': 'CrimeDate',  'OFFENSE_DESCRIPTION': 'Description', 'REPORTING_AREA': 'Neighborhood'}, inplace=True)

la_clean.rename(columns={'Crm.Cd': 'CrimeCode', 'lat': 'Latitude', 'long': 'Longitude', 'DATE.OCC': 'CrimeDate', 'AREA.NAME': 'Neighborhood', 'CrmCd.Desc': 'Description', 'Crm.Cd': 'CrimeCode', 'Location.1': 'Location'}, inplace=True)

bal_clean.rename(columns={'Location 1': 'Location'}, inplace=True)

In [147]:
bo_clean.columns.tolist()


['CrimeCode',
 'Description',
 'DISTRICT',
 'CrimeDate',
 'Year',
 'Latitude',
 'Longitude',
 'Location',
 'City']

In [148]:
la_clean.columns.tolist()

['CrimeDate',
 'CrimeCode',
 'Description',
 'Location',
 'Latitude',
 'Longitude',
 'Year',
 'City']

In [149]:
#remove all string variables; need lat and long, 

In [150]:
bal_clean.columns.tolist()

['CrimeDate',
 'CrimeCode',
 'Description',
 'Longitude',
 'Latitude',
 'Location',
 'City',
 'Year']

In [151]:
df_merged=pd.concat([bo_clean,la_clean,bal_clean])
df_merged.head()

,CrimeCode,Description,DISTRICT,CrimeDate,Year,Latitude,Longitude,Location,City
0,619,LARCENY ALL OTHERS,D14,2018-09-02 13:00:00,2018,42.357791,-71.139371,"(42.35779134, -71.13937053)",1
1,1402,VANDALISM,C11,2018-08-21 00:00:00,2018,42.306821,-71.060300,"(42.30682138, -71.06030035)",1
2,3410,TOWED MOTOR VEHICLE,D4,2018-09-03 19:27:00,2018,42.346589,-71.072429,"(42.34658879, -71.07242943)",1
3,3114,INVESTIGATE PROPERTY,D4,2018-09-03 21:16:00,2018,42.334182,-71.078664,"(42.33418175, -71.07866441)",1
4,3114,INVESTIGATE PROPERTY,B3,2018-09-03 21:05:00,2018,42.275365,-71.090361,"(42.27536542, -71.09036101)",1


In [152]:
df_merged.nunique()          

CrimeCode         425
Description       406
DISTRICT           12
CrimeDate      221220
Year                7
Latitude        36787
Longitude       40616
Location       181718
City                3
dtype: int64